In [ ]:
import os
import pickle
from datetime import datetime, timedelta
import pandas as pd
from gnn_package import training, preprocessing
from private_uoapi import (
    LightsailWrapper,
    LSAuth,
    LSConfig,
    DateRangeParams,
    convert_to_dataframe,
)
from gnn_package import paths

In [ ]:
raw_file_name = "test_data_1mnth.pkl"
raw_dir = paths.RAW_TIMESERIES_DIR
raw_file_path = os.path.join(raw_dir, raw_file_name)
print("Currently loading data from: ", raw_file_name)

preprocessed_file_name = "data_loaders_test_1yr.pkl"
preprocessed_dir = paths.PREPROCESSED_TIMESERIES_DIR
preprocessed_file_path = os.path.join(preprocessed_dir, preprocessed_file_name)
print("Currently saving data to: ", preprocessed_file_name)

In [ ]:
data_loaders = training.preprocess_data(
    data_file=raw_file_path,
    graph_prefix="25022025_test",
    window_size=24,  # 24 time steps as input
    horizon=6,  # Predict 6 steps ahead (1.5 hours with 15-min data)
    batch_size=32,
    standardize=True,
)

# get processed data
if os.path.exists(preprocessed_file_path):
    with open(preprocessed_file_path, "rb") as f:
        data_loaders = pickle.load(f)

else:
    with open(preprocessed_file_path, "wb") as f:
        pickle.dump(data_loaders, f)

In [ ]:
data_loaders.keys()

In [ ]:
data_loaders["train_loader"]

In [ ]:
# Train the model
results = training.train_model(
    data_loaders=data_loaders,
    input_dim=1,  # Traffic count is a single value
    hidden_dim=64,  # Size of hidden layers
    num_epochs=20,
    patience=3,  # Early stopping after 10 epochs with no improvement
)

# Save the trained model
from torch import save

save(results["model"].state_dict(), "stgnn_model.pth")